# Watermass transformation in the CDW density range



Watermass transformation rates are defined as:

\begin{eqnarray}
    \Omega &=& \frac{\partial}{\partial \sigma} \int \int \int \frac{D \sigma'}{D t} dV, \\
    \Omega(\sigma, t)_{i,j} &=& A_c \frac{1}{\Delta \sigma} \sum^{N_z} \bigg (  \frac{D \sigma}{D t}  h_c \Delta z_f \delta(\sigma - \sigma') \bigg ),
\end{eqnarray}

$$ \frac{D \sigma}{Dt} = \frac{\partial \sigma}{\partial \theta} \dot{\theta} + \frac{\partial \sigma}{\partial S}\dot{S} $$

$ \alpha = -\frac{1}{\rho} \frac{\partial \sigma}{\partial \theta}$ and $\beta = \frac{1}{\rho} \frac{\partial \sigma}{\partial S_A}$

$$\dot{\theta} = \frac{D \theta}{Dt} = G^\theta_{hdiff} + G^\theta_{vdiff} + G^\theta_{surf} + G^\theta_{SW}$$

$$\dot{S} = \frac{D S}{Dt} = G^S_{hdiff} + G^S_{vdiff} + G^S_{surf}$$


Since we have the theta and salinity advective terms available as online outputs at monthly frequency, we can use that to directly obtain the 

$$\dot{\theta} = \frac{\delta \theta}{\Delta t} + \frac{\delta (u\theta \Delta z \Delta y)}{\Delta x \Delta y \Delta z} + \frac{\delta (v\theta \Delta z \Delta x)}{\Delta x \Delta y \Delta z}$$ 

Where $\delta$ is the difference operator.



In [1]:
%config Completer.use_jedi = False

In [65]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import matplotlib.pyplot as plt
import cmocean as cm
import xarray as xr
import numpy as np
import IPython.display
import cosima_cookbook as cc
import pandas as pd
import gsw

In [12]:
session = cc.database.create_session()

In [3]:
expList = cc.querying.get_experiments(session)

In [99]:
expList.loc[expList["experiment"].str.contains("01deg_jra55v140_iaf_cycle4")]

,experiment,ncfiles
94,01deg_jra55v140_iaf_cycle4,131990
156,01deg_jra55v140_iaf_cycle4_jra55v150_extension,13422
184,01deg_jra55v140_iaf_cycle4_rerun_from_1983,172
185,01deg_jra55v140_iaf_cycle4_rerun_from_1986,86
188,01deg_jra55v140_iaf_cycle4_rerun_from_1980,2582


In [13]:
experiment = "01deg_jra55v140_iaf_cycle4"

In [101]:
fileList = cc.querying.get_ncfiles(session, experiment)

In [14]:
varList = cc.querying.get_variables(session, experiment=experiment, frequency="1 monthly")

In [91]:
varList["frequency"].unique()

array([None, '1 daily', '1 monthly', 'static'], dtype=object)

In [79]:
varList.loc[varList["long_name"].str.lower().str.contains("density")]#.loc[163, "ncfile"]

,name,long_name,units,frequency,ncfile,cell_methods,# ncfiles,time_start,time_end
82,mld,mixed layer depth determined by density criteria,m,1 monthly,output991/ocean/ocean-2d-mld-1-monthly-mean-ym...,time: mean,732,1958-01-01 00:00:00,2019-01-01 00:00:00
83,mld_max,mixed layer depth determined by density criteria,m,1 monthly,output991/ocean/ocean-2d-mld-1-monthly-max-ym_...,time: max,732,1958-01-01 00:00:00,2019-01-01 00:00:00
85,neutral,neutral density,kg/m^3,1 monthly,output991/ocean/ocean-3d-ty_trans_nrho_submeso...,None,732,1958-01-01 00:00:00,2019-01-01 00:00:00
86,neutralrho_edges,neutral density edges,kg/m^3,1 monthly,output991/ocean/ocean-3d-ty_trans_nrho_submeso...,None,732,1958-01-01 00:00:00,2019-01-01 00:00:00
92,pot_rho_0,potential density referenced to 0 dbar,kg/m^3,1 monthly,output991/ocean/ocean-3d-pot_rho_0-1-monthly-m...,time: mean,732,1958-01-01 00:00:00,2019-01-01 00:00:00
93,pot_rho_2,potential density referenced to 2000 dbar,kg/m^3,1 monthly,output991/ocean/ocean-3d-pot_rho_2-1-monthly-m...,time: mean,732,1958-01-01 00:00:00,2019-01-01 00:00:00
95,potrho,potential density,kg/m^3,1 monthly,output991/ocean/ocean-3d-tx_trans_rho-1-monthl...,None,1464,1958-01-01 00:00:00,2019-01-01 00:00:00
96,potrho_edges,potential density edges,kg/m^3,1 monthly,output991/ocean/ocean-3d-tx_trans_rho-1-monthl...,None,1464,1958-01-01 00:00:00,2019-01-01 00:00:00


In [18]:
average_DT = cc.querying.getvar(experiment, "average_DT", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

0.3.0


In [80]:
potrho = cc.querying.getvar(experiment, "potrho", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

In [41]:
sw_ocean = cc.querying.getvar(experiment, "sw_ocean", session, frequency="1 monthly")

In [19]:
temp = cc.querying.getvar(experiment, "temp", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

temp = temp.sel(yt_ocean = slice(-90, -60))

temp_xflux_adv = cc.querying.getvar(experiment, "temp_xflux_adv", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

temp_xflux_adv = temp_xflux_adv.sel(yt_ocean = slice(-90, -60))

temp_yflux_adv = cc.querying.getvar(experiment, "temp_yflux_adv", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")

temp_yflux_adv = temp_yflux_adv.sel(yu_ocean = slice(-90, -60))

salt = cc.querying.getvar(experiment, "salt", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")
salt = salt.sel(yt_ocean = slice(-90, -60))

salt_xflux_adv = cc.querying.getvar(experiment, "salt_xflux_adv", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")
salt_xflux_adv = salt_xflux_adv.sel(yt_ocean = slice(-90, -60))

salt_yflux_adv = cc.querying.getvar(experiment, "salt_yflux_adv", session, start_time="1990-01-01", end_time="2019-01-01", frequency="1 monthly")


KeyError: "'yt_ocean' is not a valid dimension or coordinate for Dataset with dimensions FrozenMappingWarningOnValuesAccess({'xt_ocean': 3600, 'yu_ocean': 2700, 'st_ocean': 75, 'time': 349})"

In [20]:
salt_yflux_adv = salt_yflux_adv.sel(yu_ocean = slice(-90, -60))

In [21]:
ds = xr.Dataset(coords = {"yt_ocean":(["yt_ocean"], temp.yt_ocean.values, {'axis': 'Y', 'c_grid_axis_shift': None}),
                          "yu_ocean":(["yu_ocean"], temp_yflux_adv.yu_ocean.values, {'axis': 'Y', 'c_grid_axis_shift': -0.5}),
                          "xt_ocean":(["xt_ocean"], temp.xt_ocean.values, {"axis":"X", "c_grid_axis_shift":None}),
                          "xu_ocean":(["xu_ocean"], temp_xflux_adv.xu_ocean.values, {"axis":"X", "c_grid_axis_shift": -0.5})
                         })

In [22]:
delta_lat_t2t = np.diff(ds["yt_ocean"])
delta_lat_t2t = np.append(delta_lat_t2t, delta_lat_t2t[-1])

delta_lat_u2u = np.diff(ds["yu_ocean"].values)
delta_lat_u2u = np.append(delta_lat_u2u, delta_lat_u2u[-1])

In [23]:
Re = 6370e3 # Radius of the earth in meters

In [24]:
delta_y_t2t = np.abs(Re * np.deg2rad(delta_lat_t2t)) # delta(lat) converted to distance in meters 
delta_y_u2u = np.abs(Re * np.deg2rad(delta_lat_u2u)) # delta(lat) converted to distance in meters 

In [25]:
delta_lon_t2t = np.diff(ds["xt_ocean"])
delta_lon_t2t = np.append(delta_lon_t2t, delta_lon_t2t[-1])

In [26]:
delta_lon_u2u = np.diff(ds["xu_ocean"])
delta_lon_u2u = np.append(delta_lon_u2u, delta_lon_u2u[-1])

In [27]:
delta_x_t2t = np.zeros((ds["yt_ocean"].shape[-1], ds["xu_ocean"].shape[-1]))
for i in range(len(ds["yt_ocean"].values)):
    delta_x_t2t[i] = np.abs(Re * np.cos( np.deg2rad(ds["yt_ocean"].values[i]) ) * np.deg2rad(delta_lon_t2t))

In [28]:
delta_x_u2u = np.zeros((ds["yt_ocean"].shape[-1], ds["xt_ocean"].shape[-1]))
for i in range(len(ds["yt_ocean"].values)):
    delta_x_u2u[i] = np.abs(Re * np.cos( np.deg2rad(ds["yt_ocean"].values[i]) ) * np.deg2rad(delta_lon_u2u))

In [36]:
st_ocean = temp.st_ocean
delta_s_t2t = np.diff(st_ocean)
delta_s_t2t = np.append(delta_s_t2t, delta_s_t2t[-1])

In [42]:
delta_s_w2w = np.diff(sw_ocean)
delta_s_w2w = np.append(delta_s_w2w, delta_s_w2w[-1])

In [29]:
time_vars = temp.time

In [44]:
ds = xr.Dataset(coords = {"yt_ocean":(["yt_ocean"], temp.yt_ocean.values, {'axis': 'Y', 'c_grid_axis_shift': None}),
                          "yu_ocean":(["yu_ocean"], temp_yflux_adv.yu_ocean.values, {'axis': 'Y', 'c_grid_axis_shift': -0.5}),
                          "xt_ocean":(["xt_ocean"], temp.xt_ocean.values, {"axis":"X", "c_grid_axis_shift":None}),
                          "xu_ocean":(["xu_ocean"], temp_xflux_adv.xu_ocean.values, {"axis":"X", "c_grid_axis_shift": -0.5}),
                          "st_ocean":(["st_ocean"], temp.st_ocean.values, {"axis":"Z", "c_grid_axis_shift": None}),
                          "sw_ocean":(["sw_ocean"], sw_ocean.values, {"axis":"Z", "c_grid_axis_shift": -0.5}),

                          "delta_s_t2t":(["sw_ocean"], delta_s_t2t, {"axis":"Z", "c_grid_axis_shift":-0.5}),
                          "delta_s_w2w":(["st_ocean"], delta_s_w2w, {"axis":"Z", "c_grid_axis_shift":None}),
                          "delta_y_t2t":(["yu_ocean"], delta_y_t2t, {'axis': 'Y', 'c_grid_axis_shift': -0.5}),
                          "delta_y_u2u":(["yt_ocean"], delta_y_u2u, {'axis': 'Y', 'c_grid_axis_shift': None}),
                          "delta_x_t2t":(["yt_ocean", "xu_ocean"], delta_x_t2t),
                          "delta_x_u2u":(["yt_ocean", "xt_ocean"], delta_x_u2u),
                         })

In [45]:
from xgcm import Grid

In [46]:
xgrid = Grid(ds, 
            metrics={("X",):["delta_x_t2t", "delta_x_u2u"],
                     ("Y",):["delta_y_t2t", "delta_y_u2u"],
                     ("Z"):["delta_s_t2t", "delta_s_w2w"]
            })

In [47]:
volume_cell_t = ds["delta_x_u2u"] * ds["delta_y_u2u"] * ds["delta_s_w2w"]

In [55]:
xgrid.diff(temp_yflux_adv.sel(time = slice(ts, te)), "Y")

<xarray.DataArray 'temp_yflux_adv' (time: 24, st_ocean: 75, yt_ocean: 490,
                                    xt_ocean: 3600)>
dask.array<transpose, shape=(24, 75, 490, 3600), dtype=float32, chunksize=(1, 19, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -60.15 -60.1 -60.05
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
Dimensions without coordinates: time

In [54]:
ts, te = "1990-01-01", "1992-01-01"

In [60]:
temp.sel(time=slice(ts,te))

<xarray.DataArray 'temp' (time: 24, st_ocean: 75, yt_ocean: 490, xt_ocean: 3600)>
dask.array<getitem, shape=(24, 75, 490, 3600), dtype=float32, chunksize=(1, 19, 135, 180), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.8 -279.7 ... 79.75 79.85 79.95
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -60.15 -60.1 -60.05
  * st_ocean  (st_ocean) float64 0.5413 1.681 2.94 ... 5.511e+03 5.709e+03
  * time      (time) datetime64[ns] 1990-01-16T12:00:00 ... 1991-12-16T12:00:00
Attributes:
    long_name:      Conservative temperature
    units:          K
    valid_range:    [-10. 500.]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    standard_name:  sea_water_conservative_temperature
    ncfiles:        ['/g/data/cj50/access-om2/raw-output/access-om2-01/01deg_...
    contact:        Andrew Kiss
    email:          andrew.kiss@anu.edu.au
    created:        2022-04-27
    description:    0.1 degree ACCESS-OM2 global model configuration under in...
    notes:          Run configuration and history: https://github.com/COSIMA/...

In [69]:
ts, te = "1990-01-01", "1992-01-01" # time start and end

Cp, rho0 = 4e3, 1035.0

Dtemp_Dt = temp.sel(time=slice(ts,te)).chunk((24, 19, 135, 180)).differentiate("time", datetime_unit="s") + xgrid.diff(temp_xflux_adv.sel(time = slice(ts, te)), "X") / (volume_cell_t*Cp*rho0) + \
           xgrid.diff(temp_yflux_adv.sel(time = slice(ts, te)), "Y") / (volume_cell_t*Cp*rho0)

In [70]:
Dsalt_Dt = salt.sel(time=slice(ts,te)).chunk((24, 19, 135, 180)).differentiate("time", datetime_unit="s") + xgrid.diff(salt_xflux_adv.sel(time = slice(ts, te)), "X") / (rho0*volume_cell_t) + \
           xgrid.diff(salt_yflux_adv.sel(time = slice(ts, te)), "Y") / (rho0*volume_cell_t)

In [71]:
DsaltAbs_Dt = gsw.SA_from_SP(Dsalt_Dt, st_ocean, Dsalt_Dt.xt_ocean, Dsalt_Dt.yt_ocean)

In [73]:
absSalt = gsw.SA_from_SP(salt, st_ocean, salt.xt_ocean, salt.yt_ocean)

In [74]:
alpha = gsw.alpha(absSalt, temp, st_ocean)

In [75]:
beta = gsw.beta(absSalt, temp, st_ocean)

In [76]:
Dsigma_Dt_temp = -rho0 * alpha * Dtemp_Dt

In [77]:
Dsigma_Dt_salt = rho0 * beta * DsaltAbs_Dt

In [78]:
Dsigma_Dt = Dsigma_Dt_temp + Dsigma_Dt_salt

# Experimental and trial work below

In [7]:
x = np.arange(0, 51, 5)
delta_x = np.diff(x)
y = np.linspace(0, 10, x.shape[-1])
delta_y = np.diff(y)

u = np.random.randn(delta_x.shape[-1])
theta = np.random.randn(delta_x.shape[-1]) + 10

In [10]:
np.diff(u*theta*delta_y) / (delta_x*delta_y)[:-1]

array([-4.48262572, -1.4165097 ,  4.2525832 , -2.62312913, -3.42340953,
        2.75148562, -2.05581626, -2.02625185,  6.77570832])

In [11]:
np.diff(u*theta)/delta_x[:-1]

array([-4.48262572, -1.4165097 ,  4.2525832 , -2.62312913, -3.42340953,
        2.75148562, -2.05581626, -2.02625185,  6.77570832])